## Import data for classification

In [21]:
# packages
import os
import pandas as pd
import math
from scipy import io
import numpy as np
from numpy import squeeze
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import zero_one_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot') 

In [22]:
allData = pd.read_csv('sessionTrialTable.csv')

In [23]:
allData.keys()

Index(['animalName', 'sessionDate', 'trialNum', 'totalCellNum', 'gender',
       'genotype', 'weight_g', 'ketamine_day', 'correlationScore',
       'lickAccuracy', 'lickNumber', 'avgFR', 'avgSingleCellVariance',
       'varianceFR', 'avgTrialSpeed', 'varianceSpeed', 'medianCellDepth',
       'timeSinceKetamine', 'ketamineAdministered'],
      dtype='object')

In [24]:
allData

,animalName,sessionDate,trialNum,totalCellNum,gender,genotype,weight_g,ketamine_day,correlationScore,lickAccuracy,lickNumber,avgFR,avgSingleCellVariance,varianceFR,avgTrialSpeed,varianceSpeed,medianCellDepth,timeSinceKetamine,ketamineAdministered
0,G1,190817,51,109,F,WT,16.6,1,0.163558,0.000000,0,5.727043,61.871768,0.251715,14.619883,3.636796,682.483910,-1001.54,0
1,G1,190817,52,109,F,WT,16.6,1,0.312514,0.666667,9,5.913846,96.169278,0.221846,11.185682,3.956635,682.483910,-974.18,0
2,G1,190817,53,109,F,WT,16.6,1,0.318432,0.736842,19,5.604571,87.453155,0.164644,22.148394,5.947643,682.483910,-938.42,0
3,G1,190817,54,109,F,WT,16.6,1,0.320720,0.818182,11,5.773666,102.663262,0.166935,23.174971,6.681968,682.483910,-920.36,0
4,G1,190817,55,109,F,WT,16.6,1,0.372832,0.777778,9,6.096222,112.425131,0.400779,22.148394,6.276639,682.483910,-903.10,0
5,G1,190817,56,109,F,WT,16.6,1,0.262961,1.000000,7,4.999970,71.538441,0.389813,22.598870,7.250668,682.483910,-885.04,0
6,G1,190817,57,109,F,WT,16.6,1,0.248806,0.928571,14,5.071614,86.981186,0.109385,21.668472,7.237725,682.483910,-867.34,0
7,G1,190817,58,109,F,WT,16.6,1,0.317442,0.714286,7,5.695040,92.593727,0.248882,21.929825,7.896322,682.483910,-848.88,0
8,G1,190817,59,109,F,WT,16.6,1,0.327913,0.266667,15,5.699893,89.224634,0.439541,21.505376,7.091914,682.483910,-830.64,0
9,G1,190817,60,109,F,WT,16.6,1,0.274664,0.615385,13,5.330778,84.070698,0.280866,23.310023,8.088439,682.483910,-812.04,0


In [25]:
# Check size information
print("num_cols =",len(allData.keys()))
print("num_rows =",len(allData))

# Check for duplicate rows
print("num_dup =",np.sum(pd.DataFrame.duplicated(allData)))

num_cols = 19
num_rows = 5000
num_dup = 0


In [26]:
# Check for NaNs and see where they are coming from
np.sum(pd.isna(allData))

animalName               0
sessionDate              0
trialNum                 0
totalCellNum             0
gender                   0
genotype                 0
weight_g                 0
ketamine_day             0
correlationScore         0
lickAccuracy             0
lickNumber               0
avgFR                    1
avgSingleCellVariance    1
varianceFR               3
avgTrialSpeed            0
varianceSpeed            0
medianCellDepth          0
timeSinceKetamine        0
ketamineAdministered     0
dtype: int64

In [27]:
# Remove any rows with nans
allDataNN = pd.DataFrame.dropna(allData,'index')
print("After Drop NaN")
print("num_rows =",len(allDataNN))

After Drop NaN
num_rows = 4997


In [28]:
ketBool = allDataNN['ketamineAdministered']
timeSinceKetamine = allDataNN['timeSinceKetamine']
sessionDate = allDataNN['sessionDate']
trialNum = allDataNN['trialNum']
neuralData = allDataNN[['animalName', 'totalCellNum',
       'gender', 'genotype', 'weight_g',
       'ketamine_day', 'correlationScore', 'lickAccuracy',
       'lickNumber', 'avgFR', 'avgSingleCellVariance',
       'varianceFR', 'avgTrialSpeed', 'varianceSpeed',
       'medianCellDepth']]

# Convert categorical columns
le = LabelEncoder()
neuralData_LE = neuralData.copy()
neuralData_LE['animalName'] = le.fit_transform(neuralData_LE['animalName'])
neuralData_LE['gender'] = le.fit_transform(neuralData_LE['gender'])
neuralData_LE['genotype'] = le.fit_transform(neuralData_LE['genotype'])

## Augment with interaction terms and standardize

In [29]:
AugData = neuralData_LE.copy()

AugData['animalNamexCorrelationScore'] = AugData['animalName']*AugData['correlationScore']
AugData['animalNamexLickAccuracy'] = AugData['animalName']*AugData['lickAccuracy']
AugData['animalNamexLickNumber'] = AugData['animalName']*AugData['lickNumber']
AugData['animalNamexAvgFR'] = AugData['animalName']*AugData['avgFR']
AugData['animalNamexAvgSingleCellVariance'] = AugData['animalName']*AugData['avgSingleCellVariance']
AugData['animalNamexVarianceFR'] = AugData['animalName']*AugData['varianceFR']
AugData['animalNamexAvgTrialSpeed'] = AugData['animalName']*AugData['avgTrialSpeed']
AugData['animalNamexVarianceSpeed'] = AugData['animalName']*AugData['varianceSpeed']

AugData['totalCellNumxCorrelationScore'] = AugData['totalCellNum']*AugData['correlationScore']
AugData['totalCellNumxLickAccuracy'] = AugData['totalCellNum']*AugData['lickAccuracy']
AugData['totalCellNumxLickNumber'] = AugData['totalCellNum']*AugData['lickNumber']
AugData['totalCellNumxAvgFR'] = AugData['totalCellNum']*AugData['avgFR']
AugData['totalCellNumxAvgSingleCellVariance'] = AugData['totalCellNum']*AugData['avgSingleCellVariance']
AugData['totalCellNumxVarianceFR'] = AugData['totalCellNum']*AugData['varianceFR']
AugData['totalCellNumxAvgTrialSpeed'] = AugData['totalCellNum']*AugData['avgTrialSpeed']
AugData['totalCellNumxVarianceSpeed'] = AugData['totalCellNum']*AugData['varianceSpeed']

AugData['genderxCorrelationScore'] = AugData['gender']*AugData['correlationScore']
AugData['genderxLickAccuracy'] = AugData['gender']*AugData['lickAccuracy']
AugData['genderxLickNumber'] = AugData['gender']*AugData['lickNumber']
AugData['genderxAvgFR'] = AugData['gender']*AugData['avgFR']
AugData['genderxAvgSingleCellVariance'] = AugData['gender']*AugData['avgSingleCellVariance']
AugData['genderxVarianceFR'] = AugData['gender']*AugData['varianceFR']
AugData['genderxAvgTrialSpeed'] = AugData['gender']*AugData['avgTrialSpeed']
AugData['genderxVarianceSpeed'] = AugData['gender']*AugData['varianceSpeed']

AugData['genotypexCorrelationScore'] = AugData['genotype']*AugData['correlationScore']
AugData['genotypexLickAccuracy'] = AugData['genotype']*AugData['lickAccuracy']
AugData['genotypexLickNumber'] = AugData['genotype']*AugData['lickNumber']
AugData['genotypexAvgFR'] = AugData['genotype']*AugData['avgFR']
AugData['genotypexAvgSingleCellVariance'] = AugData['genotype']*AugData['avgSingleCellVariance']
AugData['genotypexVarianceFR'] = AugData['genotype']*AugData['varianceFR']
AugData['genotypexAvgTrialSpeed'] = AugData['genotype']*AugData['avgTrialSpeed']
AugData['genotypexVarianceSpeed'] = AugData['genotype']*AugData['varianceSpeed']

AugData['weight_gxCorrelationScore'] = AugData['weight_g']*AugData['correlationScore']
AugData['weight_gxLickAccuracy'] = AugData['weight_g']*AugData['lickAccuracy']
AugData['weight_gxLickNumber'] = AugData['weight_g']*AugData['lickNumber']
AugData['weight_gxAvgFR'] = AugData['weight_g']*AugData['avgFR']
AugData['weight_gxAvgSingleCellVariance'] = AugData['weight_g']*AugData['avgSingleCellVariance']
AugData['weight_gxVarianceFR'] = AugData['weight_g']*AugData['varianceFR']
AugData['weight_gxAvgTrialSpeed'] = AugData['weight_g']*AugData['avgTrialSpeed']
AugData['weight_gxVarianceSpeed'] = AugData['weight_g']*AugData['varianceSpeed']

AugData['ketamine_dayxCorrelationScore'] = AugData['ketamine_day']*AugData['correlationScore']
AugData['ketamine_dayxLickAccuracy'] = AugData['ketamine_day']*AugData['lickAccuracy']
AugData['ketamine_dayxLickNumber'] = AugData['ketamine_day']*AugData['lickNumber']
AugData['ketamine_dayxAvgFR'] = AugData['ketamine_day']*AugData['avgFR']
AugData['ketamine_dayxAvgSingleCellVariance'] = AugData['ketamine_day']*AugData['avgSingleCellVariance']
AugData['ketamine_dayxVarianceFR'] = AugData['ketamine_day']*AugData['varianceFR']
AugData['ketamine_dayxAvgTrialSpeed'] = AugData['ketamine_day']*AugData['avgTrialSpeed']
AugData['ketamine_dayxVarianceSpeed'] = AugData['ketamine_day']*AugData['varianceSpeed']

AugData['medianCellDepthxCorrelationScore'] = AugData['medianCellDepth']*AugData['correlationScore']
AugData['medianCellDepthxLickAccuracy'] = AugData['medianCellDepth']*AugData['lickAccuracy']
AugData['medianCellDepthxLickNumber'] = AugData['medianCellDepth']*AugData['lickNumber']
AugData['medianCellDepthxAvgFR'] = AugData['medianCellDepth']*AugData['avgFR']
AugData['medianCellDepthxAvgSingleCellVariance'] = AugData['medianCellDepth']*AugData['avgSingleCellVariance']
AugData['medianCellDepthxVarianceFR'] = AugData['medianCellDepth']*AugData['varianceFR']
AugData['medianCellDepthxAvgTrialSpeed'] = AugData['medianCellDepth']*AugData['avgTrialSpeed']
AugData['medianCellDepthxVarianceSpeed'] = AugData['medianCellDepth']*AugData['varianceSpeed']

# Standardize data
stdNeuralDataAug = StandardScaler().fit_transform(AugData) 

/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## Train-Test Split

In [37]:
# Set seed for reproducibility and split into train and test
X, X_ho, y, y_ho = train_test_split(stdNeuralDataAug,ketBool.values.ravel(), test_size=0.2, random_state=2019)

In [38]:
# Split for cross validation, use 10 folds
num_folds = 10
XA = np.array(X)
yA = np.array(y)
X_train = []
X_test = []
y_train = []
y_test = []
skf = StratifiedKFold(n_splits=num_folds)
for train_index, test_index in skf.split(XA, yA):
    X_train.append(XA[train_index])
    X_test.append(XA[test_index])
    y_train.append(yA[train_index])
    y_test.append(yA[test_index])

In [39]:
# Run basic log reg model on train set, check performance against train
# Run model, tuning over C_param (l2 penalty by default)
for C_param in [0.01, 0.1, 1, 10, 100, 1000, 5000, 10000, 50000]:
    model = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000,C = C_param).fit(X, y) 
    y_pred = model.predict(X)
    print(zero_one_loss(y, y_pred))

0.12009006755066298
0.09181886414811113
0.08056042031523647
0.07255441581185884
0.07080310232674503
0.07230422817112836
0.07230422817112836
0.07230422817112836
0.07230422817112836


In [40]:
C_param = 100
zo_loss = []
accuracy = []
num_folds=10
for i in range(0,num_folds):
    model = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000, C = C_param).fit(X_train[i], y_train[i]) 
    y_pred = model.predict(X_test[i])
    zo_loss.append(zero_one_loss(y_test[i],y_pred))
    accuracy.append(accuracy_score(y_test[i],y_pred))

avg_zo_loss = np.mean(zo_loss)
avg_acc = np.mean(accuracy)
print("Average zero-one loss across folds:",avg_zo_loss)
print("Average accuracy across folds:",avg_acc)

Average zero-one loss across folds: 0.07955723317020733
Average accuracy across folds: 0.9204427668297926


## Now train model on entire training set and get details

In [41]:
LRmodel = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000, C = 100).fit(X, y)

In [42]:
y_pred = model.predict(X_ho)
zo_loss_test = zero_one_loss(y_ho,y_pred)
accuracy_test =accuracy_score(y_ho,y_pred)

In [43]:
zo_loss_test

0.08699999999999997

In [44]:
accuracy_test

0.913